In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

os.environ["OPENAI_API_KEY"] = openai_api_key

tavily_api_key = os.getenv("TAVILY_API_KEY")
os.environ["TAVILY_API_KEY"] = tavily_api_key

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator, sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage

conn = sqlite3.connect("checkpont.sqlite", check_same_thread=False)
memory = SqliteSaver(conn)

In [3]:
class AgentState(TypedDict):
    task: str 
    plan: str 
    draft: str
    critique: str 
    content: List[str]
    revision_number: int 
    max_revisions: int

In [4]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-4o", temperature=0)

In [5]:
PLAN_PROMPT = """You are an expert writer tasked with writing a high level outline of an essay. \
Write such an outline for the user provided topic. Give an outline of the essay along with any relevant notes \
or instructions for the sections."""

In [6]:
WRITER_PROMPT = """You are an essay assistant tasked with writing excellent 5-paragraph essays.\
Generate the best essay possible for the user's request and the initial outline. \
If the user provides critique, respond with a revised version of your previous attempts. \
Utilize all the information below as needed: 

------

{content}"""

In [7]:
REFLECTION_PROMPT = """You are a teacher grading an essay submission. \
Generate critique and recommendations for the user's submission. \
Provide detailed recommendations, including requests for length, depth, style, etc."""

In [8]:
RESEARCH_PLAN_PROMPT = """You are a researcher charged with providing information that can \
be used when writing the following essay. Generate a list of search queries that will gather \
any relevant information. Only generate 3 queries max."""

In [9]:
RESEARCH_CRITIQUE_PROMPT = """You are a researcher charged with providing information that can \
be used when making any requested revisions (as outlined below). \
Generate a list of search queries that will gather any relevant information. Only generate 3 queries max."""

In [10]:
from pydantic import BaseModel

class Queries(BaseModel):
    queries: List[str]

In [11]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily_tool = TavilySearchResults(max_results=2)

In [12]:
def plan_node(state: AgentState):
    messages = [
        SystemMessage(content=PLAN_PROMPT),
        HumanMessage(content= state['task'])
    ]

    response = model.invoke(messages)
    return {"plan": response.content}

In [45]:
def research_plan_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_PLAN_PROMPT),
        HumanMessage(content=state['task'])
    ])
    content = state['content'] or []
    for q in queries.queries:
        response = tavily_tool.run(q)

        for r in response:
            content.append(r['content'])
    return {"content": content}

In [ ]:
example_state: AgentState = {
    "task": "Write a story about a cat",
    "plan": "1. Introduction. 2. Middle. 3. Ending.",
    "draft": "",
    "critique": "",
    "content": ["Once upon a time..."],
    "revision_number": 1,
    "max_revisions": 3,
}

research_plan_node(example_state)

In [46]:
def generation_node(state: AgentState):
    content = "\n\n".join(state['content'] or [])
    user_message = HumanMessage(
        content = f"{state['task']}\n\nHere is my plan:\n\n{state['plan']}"
    )
    messages = [
        SystemMessage(
            content= WRITER_PROMPT.format(content=content)
        ),
        user_message
    ]

    response = model.invoke(messages)
    return {
        "draft": response.content,
        "revision_number": state.get("revision_number", 1) + 1
    }

In [47]:
def reflection_node(state: AgentState):
    messages = [
        SystemMessage( content=REFLECTION_PROMPT),
        HumanMessage(content=state['draft'])
    ]

    response = model.invoke(messages)
    return {"critique": response.content}

In [49]:
def research_critique_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content= RESEARCH_CRITIQUE_PROMPT),
        HumanMessage(content= state['critique'])
    ])

    content = state['content'] or []
    for q in queries.queries:
        response = tavily_tool.run(q)
        for r in response:
            content.append(r['content'])
    
    return {"content": content}

In [50]:
def should_continue(state: AgentState):
    return state['revision_number'] <= state['max_revisions']

In [51]:
builder = StateGraph(AgentState)       

In [52]:
builder.add_node("planner",plan_node)
builder.add_node("generate",generation_node)
builder.add_node("reflect",reflection_node)
builder.add_node("research_plan",research_plan_node)
builder.add_node("research_critique",research_critique_node)

In [53]:
builder.set_entry_point("planner")

In [54]:
builder.add_conditional_edges(
    "generate",
    should_continue,
    {True: "reflect", False: END}
)

In [55]:
builder.add_edge("planner","research_plan")
builder.add_edge("research_plan","generate")

builder.add_edge("reflect","research_critique")
builder.add_edge("research_critique","generate")

In [56]:
graph = builder.compile(checkpointer=memory)

In [65]:
import json 

thread = {"configurable": {"thread_id": "1"}}
for s in graph.stream({
    'task': "what is the difference between langchain and langsmith",
    "max_revisions": 2,
    "revision_number": 1,
    "content": []
}, thread):
     print(json.dumps(s, indent=4))

{
    "planner": {
        "plan": "**Title: Understanding the Differences Between LangChain and LangSmith**\n\n**I. Introduction**\n   - Brief introduction to the importance of language processing tools in modern technology.\n   - Explanation of the growing landscape of language models and frameworks.\n   - Introduction to LangChain and LangSmith as two prominent tools in this domain.\n\n**II. Overview of LangChain**\n   - **A. Definition and Purpose**\n     - Define LangChain and its primary objectives.\n     - Discuss its role in natural language processing (NLP) and artificial intelligence (AI).\n   - **B. Key Features**\n     - Highlight the main features and capabilities of LangChain.\n     - Discuss its architecture and how it integrates with other systems.\n   - **C. Use Cases**\n     - Provide examples of applications and industries where LangChain is commonly used.\n     - Discuss its effectiveness in various NLP tasks.\n\n**III. Overview of LangSmith**\n   - **A. Definition 